Header

In [1]:
import numpy as np
import time
import healpy as hp
import matplotlib.pyplot as plt
import cupy as cp
import pysht
from pysht import get_geom
import delensalot
from delensalot import utils
from delensalot.sims.sims_lib import Xunl, Xsky, Xobs

lmax, mmax = 1024, 1024
ll = np.arange(0,lmax)
# geominfo = ('tgl',{'lmax': lmax, 'smax':3})
geominfo = ('gl',{'lmax':lmax})
# geominfo = ('cc',{'nphi':1032, 'ntheta':2058})
# geominfo = ('healpix',{'nside': 512})

synunl = Xunl(lmax=lmax, geominfo=geominfo)
synsky = Xsky(lmax=lmax, unl_lib=synunl, geominfo=geominfo)

philm = synunl.get_sim_phi(0, space='alm')
dlm = hp.almxfl(philm, np.sqrt(ll*(ll+1)))

Tunl = synunl.get_sim_unl(0, spin=0, space='alm', field='temperature')
Tsky = synsky.get_sim_sky(0, spin=0, space='map', field='temperature')

# t.constructor.spat_shape

[SHTns 3.6.6] built Feb 29 2024, 11:27:49, id: v3.6.6*,avx512,ishioka,openmp
mpisupport: True, pmisupport: False
disabling mpi
Using lenspyx alm2map
INFO:: 02-29 14:46:06:: delensalot.sims.sims_lib.__init__ - sht_solver not given, defaulting to ducc
INFO:: 02-29 14:46:06:: delensalot.sims.sims_lib.__init__ - sht_backend not given, defaulting to CPU
INFO:: 02-29 14:46:07:: delensalot.sims.sims_lib.__init__ - sht_solver not given, defaulting to ducc
INFO:: 02-29 14:46:07:: delensalot.sims.sims_lib.__init__ - sht_backend not given, defaulting to CPU


# nuFFT test

## CPU

In [8]:
"""
CPU nuFFT
"""
deflection_kwargs = {
    'dlm':dlm,
    'mmax_dlm':len(ll),
    'epsilon':1e-5,
    'verbosity':0,
    'single_prec':False,
    'planned':False}
sht_solver = 'ducc' # 'shtns'
for solver in ['finufft', 'duccnufft']:
    for backend in ['CPU']:
        for mode in ['nuFFT']:
            t = pysht.get_transformer(solver, mode, backend)

            print("Testing function synthesis_general...")
            defres = t.synthesis_general(lmax, mmax, Tsky, [0,1], 0, 1e-5, 4, mode='GRAD_ONLY', Tunl.copy(), verbose=1)
            
            print("Testing function adjoint_synthesis_general...")
            defres = t.adjoint_synthesis_general(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)

            print("Testing solver={} backend={} mode={}...".format(solver, backend, mode))
            t = pysht.get_transformer(solver, mode, backend)
            t = t(sht_solver, geominfo, deflection_kwargs)
            print("\n----Testing function gclm2lenmap...----")
            defres = t.gclm2lenmap(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)

            print("\n----Testing function lenmap2gclm...----")
            defres = t.lenmap2gclm(points=Tsky.copy(), dlm=dlm, spin=0, lmax=lmax, mmax=lmax, nthreads=4)
            
            print("\n----Testing function lensgclm...----")
            defres = t.lensgclm(Tunl.copy(), spin=0, dlm=dlm, lmax_out=lmax, nthreads=4)

            print('\n\n')

Testing function synthesis_general...


AttributeError: type object 'CPU_finufft_transformer' has no attribute 'synthesis_general'

## GPU

In [4]:
"""
GPU nuFFT
"""
deflection_kwargs = {
    'dlm':dlm,
    'mmax_dlm':len(ll),
    'epsilon':1e-5,
    'verbosity':0,
    'single_prec':False,
    'planned':False}
sht_solver = 'ducc' # shtns
for solver in ['cufinufft']:
    for backend in ['GPU']:
        for mode in ['nuFFT']:
            print("Testing solver={} backend={} mode={}...".format(solver, backend, mode))
            t = pysht.get_transformer(solver, mode, backend)
            t = t(sht_solver, geominfo, deflection_kwargs)
            # t.set_geometry(geominfo)
            print("\n----Testing function gclm2lenmap...----")
            defres = t.gclm2lenmap(gclm=Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)

            print("\n----Testing function lenmap2gclm...----")
            defres = t.lenmap2gclm(points=Tsky.copy(), dlm=dlm, spin=0, lmax=lmax, mmax=lmax, nthreads=4)
            
            print("\n----Testing function lensgclm...----")
            defres = t.lensgclm(Tunl.copy(), spin=0, lmax_out=lmax, dlm=dlm, nthreads=4)

            print("Testing function synthesis_general...")
            defres = t.synthesis_general(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)
            
            print("Testing function adjoint_synthesis_general...")
            defres = t.adjoint_synthesis_general(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)
            print('\n\n')

Testing solver=cufinufft backend=GPU mode=nuFFT...
deflection std is 2.57e+00 amin

----Testing function gclm2lenmap...----
deflection std is 2.57e+00 amin
build_angles

----Testing function lenmap2gclm...----
Nu2u:
  nthreads=4, grid=(2056x2058), oversampled grid=(3200x3200), supp=10, eps=1.000000000000000e-07
  npoints=2214000
  memory overhead: 8.247792720794678e-03GB (index) + 1.525878906250000e-01GB (oversampled grid)

Total wall clock time for nu2u: 0.1884s
|
+- nu2u proper          : 92.67% (0.1746s)
|  |
|  +- spreading            : 62.15% (0.1085s)
|  +- FFT                  : 28.51% (0.0498s)
|  +- zeroing grid         :  5.13% (0.0090s)
|  +- grid correction      :  4.19% (0.0073s)
|  +- allocating grid      :  0.01% (0.0000s)
|  
+- building index       :  7.09% (0.0134s)
+- correction factors   :  0.16% (0.0003s)
+- parameter calculation:  0.01% (0.0000s)

----Testing function lensgclm...----





# SHT test

## CPU

In [17]:

"""
CPU SHT
"""
for solver in ['shtns', 'ducc']:
    for backend in ['CPU']:
        for mode in ['SHT']:
            print("Testing solver={} backend={} mode={}...".format(solver, backend, mode))
            t = pysht.get_transformer(solver, mode, backend)(geominfo)

            print("Testing function synthesis_general...")
            defres = t.synthesis(Tunl.copy(), lmax=lmax, mmax=lmax, spin=0, nthreads=4)
            
            print("Testing function adjoint_synthesis_general...")
            if solver == 'shtns':
                m = np.random.random(size=t.constructor.spat_shape)
            else:
                m = Tsky
            defres = t.adjoint_synthesis(m.copy(), lmax=lmax, mmax=lmax, spin=0, nthreads=4)
            print('\n\n')

Testing solver=shtns backend=CPU mode=SHT...


Testing function synthesis_general...
Testing function adjoint_synthesis_general...



Testing solver=ducc backend=CPU mode=SHT...
Testing function synthesis_general...
Testing function adjoint_synthesis_general...





In [2]:

"""
GPU SHT
"""
for solver in ['shtns']:
    for backend in ['GPU']:
        for mode in ['SHT']:
            print("Testing solver={} backend={} mode={}...".format(solver, backend, mode))
            t = pysht.get_transformer(solver, mode, backend)(geominfo)

            print("Testing function synthesis_general...")
            defres = t.synthesis(Tunl.copy(), lmax=lmax, mmax=lmax, spin=0, nthreads=4)
            
            print("Testing function adjoint_synthesis_general...")
            if solver == 'shtns':
                m = np.random.random(size=t.constructor.spat_shape)
            else:
                m = Tsky
            defres = t.adjoint_synthesis(m.copy(), lmax=lmax, mmax=lmax, spin=0, nthreads=4)
            print('\n\n')

Testing solver=shtns backend=GPU mode=SHT...
Testing function synthesis_general...
Testing function adjoint_synthesis_general...



